In [ ]:
!gcloud dataproc clusters list --region us-central1

In [ ]:
!pip install -q google-cloud-storage==1.43.0
!pip install -q graphframes

In [ ]:
import pyspark
import sys
from collections import Counter, OrderedDict, defaultdict
import itertools
from itertools import islice, count, groupby
import pandas as pd
import os
import re
from operator import itemgetter
import nltk
from nltk.stem.porter import *
from nltk.corpus import stopwords
from time import time
from pathlib import Path
import pickle
import pandas as pd
from google.cloud import storage
import builtins
import math

import hashlib
def _hash(s):
    return hashlib.blake2b(bytes(s, encoding='utf8'), digest_size=5).hexdigest()

nltk.download('stopwords')

In [ ]:
!ls -l /usr/lib/spark/jars/graph*


In [ ]:
from pyspark.sql import *
from pyspark.sql.functions import *
from pyspark import SparkContext, SparkConf, SparkFiles
from pyspark.sql import SQLContext
from graphframes import *

In [ ]:
spark

In [ ]:
bucket_name = '207044777' 
full_path = f"gs://{bucket_name}/"
paths=[]

client = storage.Client()
blobs = client.list_blobs(bucket_name)
for b in blobs:
    if b.name != 'graphframes.sh':
        paths.append(full_path+b.name)

In [ ]:
parquetFile = spark.read.parquet(*paths)
doc_text_pairs = parquetFile.select("text", "id").rdd

In [ ]:
parquetFile.count()


In [ ]:
%cd -q /home/dataproc
!ls inverted_index_gcp.py

In [ ]:
sc.addFile("/home/dataproc/inverted_index_gcp.py")
sys.path.insert(0,SparkFiles.getRootDirectory())

In [ ]:
from inverted_index_gcp import InvertedIndex


In [ ]:
english_stopwords = frozenset(stopwords.words('english'))
corpus_stopwords = ["category", "references", "also", "external", "links", 
                    "may", "first", "see", "history", "people", "one", "two", 
                    "part", "thumb", "including", "second", "following", 
                    "many", "however", "would", "became"]

all_stopwords = english_stopwords.union(corpus_stopwords)
RE_WORD = re.compile(r"""[\#\@\w](['\-]?\w){2,24}""", re.UNICODE)
stemmer=PorterStemmer()

In [ ]:
def tokenizer(text):
    tokens = [token.group() for token in RE_WORD.finditer(text.lower())]
    tokens = [stemmer.stem(t) for t  in tokens if t not in all_stopwords]
    return tokens
    
def word_count(text, id):
  tokens = tokenizer(text)
  token_counts = Counter(tokens)
  return [(token,(id,count)) for token,count in token_counts.items()] 

def reduce_word_counts(unsorted_pl):
  return sorted(unsorted_pl,key=itemgetter(0))

def calculate_df(postings):
  return postings.map(lambda x: (x[0], len(x[1])))

def partition_postings_and_write(postings):
  return postings.map(lambda x: (token2bucket_id(x[0]), x)).groupByKey().map(lambda x : InvertedIndex.write_a_posting_list(x,bucket_name))

In [ ]:
doc_wordlist_pairs = doc_text_pairs.map(lambda x: (x[1],len(tokenizer(x[0]))))

DL = doc_wordlist_pairs.collectAsMap()
N=len(DL)
AVGDL = builtins.sum(DL.values())/N

In [ ]:
def calc_idf(term):
        n_ti = w2df_dict[term]
        return math.log(1 + (N - n_ti + 0.5) / (n_ti + 0.5))
                            

def bm25_score(term, pls):
        new_pls=[]
        k1=1.3
        b=0.1
        for doc_id,tf in pls:
            doc_len = DL[doc_id] 
            if term in w2df_dict.keys():
                numerator = calc_idf(term) * tf * (k1 + 1)
                denominator = tf + k1 * (1 - b + b * doc_len / AVGDL)
                score = builtins.round((numerator / denominator),3)
                new_pls.append((doc_id,int(score*10**3)))
        return (term,sorted(new_pls,key=lambda x: x[1]))

In [ ]:
# time the index creation time
t_start = time()
# word counts map
word_counts = doc_text_pairs.flatMap(lambda x: word_count(x[0], x[1]))
postings = word_counts.groupByKey().mapValues(reduce_word_counts)
# filtering postings and calculate df
postings_filtered = postings.filter(lambda x: len(x[1])>50)
w2df = calculate_df(postings_filtered)
w2df_dict = w2df.collectAsMap()

In [ ]:
postings_bm25 = postings_filtered.map(lambda x: bm25_score(x[0],x[1]))

In [ ]:
_ = partition_postings_and_write(postings_bm25_sort).collect()

In [ ]:
# collect all posting lists locations into one super-set
super_posting_locs = defaultdict(list)
for blob in client.list_blobs(bucket_name, prefix='postings_gcp_body_bm25_stem'):
  if not blob.name.endswith("pickle"):
    continue
  with blob.open("rb") as f:
    posting_locs = pickle.load(f)
    for k, v in posting_locs.items():
      super_posting_locs[k].extend(v)

In [ ]:
# Create inverted index instance
inverted = InvertedIndex()
# Adding the posting locations dictionary to the inverted index
inverted.posting_locs = super_posting_locs
# Add the token - df dictionary to the inverted index
inverted.df = w2df_dict
# write the global stats out
inverted.write_index('.', 'index')
# upload to gs
index_src = "index.pkl"
index_dst = f'gs://{bucket_name}/postings_gcp_body_bm25_stem/{index_src}'
!gsutil cp $index_src $index_dst

In [ ]:
!gsutil ls -lh $index_dst